In [1]:
import os
import numpy as np
import torch
import torchvision
from torchvision import datasets ,models,transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# check cude is available or not
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load and Transform our Data

In [3]:
data_dir = 'flower_photos'

In [4]:
# define training and test data directories
train_dir = os.path.join(data_dir,'train/')
test_dir = os.path.join(data_dir , 'test/')

classes = ['daisy','dandelion','roses','sunflowers','tulips']

### Transforming the data

In [7]:
data_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.ToTensor()])
train_data = datasets.ImageFolder(train_dir, transform=data_transform)
test_data = datasets.ImageFolder(test_dir, transform=data_transform)

# print out some data stats
print('Num training images: ', len(train_data))
print('Num test images: ', len(test_data))

Num training images:  3130
Num test images:  540


### DataLoaders and Data Visualization

In [11]:
# define dataloader parameters
batch_size = 20
num_workers=0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)


## Define the model

To define a model for training we'll follow these steps:

1. Load in a pre-trained VGG16 model
2. "Freeze" all the parameters, so the net acts as a fixed feature extractor
3. Remove the last layer
4. Replace the last layer with a linear classifier of our own
###  Freezing simply means that the parameters in the pre-trained model will not change during training.

In [15]:
#load the pretrained model from pytorch
vgg16 = models.vgg16(pretrained=True)

# Print out the model structure
print(vgg16)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
print(vgg16.classifier[6].in_features)
print(vgg16.classifier[6].out_features)

4096
1000


In [17]:
#freeze training for all "features" layers
for param in vgg16.features.parameters():
    param.requires_grad = False

### Final Classifier Layer

Once you have the pre-trained feature extractor, you just need to modify and/or add to the final, fully-connected classifier layers. In this case, we suggest that you repace the last layer in the vgg classifier group of layers.

This layer should see as input the number of features produced by the portion of the network that you are not changing, and produce an appropriate number of outputs for the flower classification task.

### You can access any layer in a pretrained network by name and (sometimes) number, i.e. vgg16.classifier[6] is the sixth layer in a group of layers named "classifier".

In [20]:
import torch.nn as nn

n_inputs = vgg16.classifier[6].in_features

# add last linear layer (n_inputs -> 5 flower classes)
# new layers automatically have requires_grad = True

last_layer = nn.Linear(n_inputs, len(classes))

vgg16.classifier[6] = last_layer

# model to device
vgg16.to(device)


# check to see that your last layer produces the expected number of outputs
print(vgg16.classifier[6].out_features)
#print(vgg16)

5


### Specifying Loss function and Optimizer

## only train the classifier parameters ,features parameters are frozen

#### in optimizer for back propagation 

In [21]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001)

### Training

In [23]:
# number of epochs to train the model
n_epochs = 2

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    # model by default is set to train
    for idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = vgg16(data)
        
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss 
        train_loss += loss.item()
        
        if idx % 20 == 19:    # print training loss every specified number of mini-batches
            print('Epoch %d, Batch %d loss: %.16f' %
                  (epoch, idx + 1, train_loss / 20))
            train_loss = 0.0

Epoch 1, Batch 20 loss: 1.5212907016277313
Epoch 1, Batch 40 loss: 1.3896093189716339
Epoch 1, Batch 60 loss: 1.1862610220909118
Epoch 1, Batch 80 loss: 1.1190831691026688
Epoch 1, Batch 100 loss: 1.0770229220390319
Epoch 1, Batch 120 loss: 0.9912109941244125
Epoch 1, Batch 140 loss: 0.9197895050048828
Epoch 2, Batch 20 loss: 0.8587419986724854
Epoch 2, Batch 40 loss: 0.8469494223594666
Epoch 2, Batch 60 loss: 0.8397821396589279
Epoch 2, Batch 80 loss: 0.8569969236850739
Epoch 2, Batch 100 loss: 0.7360910311341285
Epoch 2, Batch 120 loss: 0.7508710920810699
Epoch 2, Batch 140 loss: 0.7467984184622765


### Testing

Below you see the test accuracy for each flower class.

In [28]:
# track test loss 
# over 5 flower classes
test_loss = 0.0
correct = 0 #list(0. for i in range(5))
total =0 # list(0. for i in range(5))

vgg16.eval() # eval mode

# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = vgg16(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()*data.size(0)
    
    # convert output probabilities to predicted class
    _, pred = torch.max(output.data, 1)    
    total += target.size(0)
    correct += (pred==target).sum().item()
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 78.33333333333333 %
